# Process the Customer Data 
1. Ingest the data into the data lakehouse bronze table: bronze_customer
2. Perform data quality checks and transform the data as required to silver table: silver_customers_clean
3. Apply changes to the customer data - silver_customers 

In [0]:
  %sql
  create or refresh streaming table circuitbox.default.bronze_customers
  comment 'Raw customers data ingested from source system'
  tblproperties ('quality' = 'bronze')
  as 
  select *,
  _metadata.file_path as file_path,
  current_timestamp as ingestion_timestamp
  from cloud_files(
    '/Volumes/circuitbox/landing/operational_data/customers/',
    'json',
    map('cloudFiles.inferColumnTypes', 'true')
  )

### Perform Data quality checks and transform the data as required - silver_customers_clean

##### Data Quality Rules 
- Fail if customer_id is null
- Drop record if customer name as null
- Warn if telephone is less then 10 characters 
- Warn if email is null
- warn if date_of_birth is before 1920

##### More transformations
- Cast date_of_birth as date
- Cast created_date as date

In [0]:
create or refresh streaming table circuitbox.silver.silver_customers_clean(
  constraint valid_customer_id expect (customer_id is not null) on violation fail update,
  constraint valid_customer_name expect (customer_name is not null) on violation drop row,
  constraint valid_telephone expect (length(telephone) >= 10),
  constraint valid_email expect (email is not null),
  constraint valid_date_of_birth expect (date_of_birth >= '1920-01-01')

)
comment 'Cleaned customer data'
tblproperties ('quality' = 'silver')
as
select customer_id,
customer_name,
cast(date_of_birth as date) as date_of_birth,
telephone,
email,
cast(created_date as date) as created_date
from stream(circuitbox.default.bronze_customers)

### Apply changes to customer data using : silver_customers

In [0]:
create or refresh streaming table circuitbox.default.silver_customers
comment 'SCD type 1 customers data'
tblproperties ('quality' = 'silver')

In [0]:
apply changes into circuitbox.default.silver_customers
from stream(circuitbox.silver.silver_customers_clean)
keys(customer_id)
sequence by created_date
stored as scd type 1; -- Default value is type 1